# Deformation of solids

**Prashant K. Jha**

*Department of Mechanical Engineering, South Dakota School of Mines and Technology, Rapid City, SD, USA*

In this tutorial, we model the deformation of elastic materials using fenics. The problems considered include:
1. Deformation of a linear elastic material modeled using small deformation theory. This results in a linear variational problem. I
2. Deformation of an orthotropic elastic material (as an example of anisotropic material). 
3. Deformation of a hyperelastic material assuming large deformation theory. The resulting partial differential equation is highly nonlinear in the displacement.

## Large deformation of hard magnetic soft materials

Consider a 3-D beam $\Omega = \{(x,y,z): x\in (0, L),\; y\in (0,W),\; z\in (0,H)\}$ as shown in the figure. The beam is fixed on the left surface (at $x=0$), $\Gamma_l = \{(x,y,z) \in \Omega: x = 0\}$, and is subjected to external traction $\boldsymbol{t}$ on the right surface (at $x=L$), $\Gamma_r = \{(x,y,z) \in \Omega: x = L\}$. The beam is also subjected to external magnetic field in y-direction.

<img src="results/hyperelastic_problem.png" style="width:400px;">

To describe the large deformation, we introduce few notations:
- $\boldsymbol{X} = (X_1, X_2, X_3) \in \Omega$ denotes the position of a point in the reference configuration (initial configuration) of solid;
- $\boldsymbol{x}(\boldsymbol{X}) = (x_1(\boldsymbol{X}), x_2(\boldsymbol{X}), x_3(\boldsymbol{X})) \in \Omega$ denotes the position of a point $\boldsymbol{X}\in \Omega$ in the current configuration of solid;
- $\boldsymbol{u}(\boldsymbol{X}) = \boldsymbol{x}(\boldsymbol{X}) - \boldsymbol{X}$ denotes the displacement of a reference point $\boldsymbol{X}$;
- $\boldsymbol{F} = \frac{\partial \boldsymbol{x}}{\partial \boldsymbol{X}} = \boldsymbol{I} + \nabla_{\boldsymbol{X}} \boldsymbol{u}(\boldsymbol{X})$ is the deformation gradient, where $\nabla_{\boldsymbol{X}} g = \frac{\partial g}{\partial \boldsymbol{X}} = (\frac{\partial g}{\partial X_1}, \frac{\partial g}{\partial X_2}, \frac{\partial g}{\partial X_3})$ denotes the gradient of a scalar function $g$ with respect to $\boldsymbol{X}$;
- $\boldsymbol{C} = \boldsymbol{F}^T \boldsymbol{F}$ is the right Cauchy-Green strain tensor, where $\boldsymbol{F}^T$ denotes the transpose of a tensor;
- $\boldsymbol{E} = (\boldsymbol{C} - \boldsymbol{I})/2$ is the Lagrangian measure of strain known as the Green strain tensor. We can show that
\begin{equation}\tag{1}
\boldsymbol{E} = \frac{1}{2}\left[\nabla_{\boldsymbol{X}} \boldsymbol{u} + \left(\nabla_{\boldsymbol{X}} \boldsymbol{u} \right)^T + \left(\nabla_{\boldsymbol{X}} \boldsymbol{u} \right)^T \left( \nabla_{\boldsymbol{X}} \boldsymbol{u} \right) \right] = \boldsymbol{e} + \frac{1}{2} \left(\nabla_{\boldsymbol{X}} \boldsymbol{u} \right)^T \left( \nabla_{\boldsymbol{X}} \boldsymbol{u} \right)\,,
\end{equation}
where $\boldsymbol{e} = \frac{1}{2}\left[\nabla_{\boldsymbol{X}} \boldsymbol{u} + \left(\nabla_{\boldsymbol{X}} \boldsymbol{u} \right)^T \right]$ is the linearized strain tensor. Thus, we see that the Green strain tensor $\boldsymbol{E}$ is a nonlinear function of displacement. 
- $\boldsymbol{\sigma}(\boldsymbol{x})$ denotes the Cauchy stress tensor defined in the deformed solid. 
- $\boldsymbol{P}(\boldsymbol{X})$ is the first Piola Kirchhoff stress tensor defined in reference configuration. We have, for $\boldsymbol{x}= \boldsymbol{x}(\boldsymbol{X})$,
\begin{equation}\tag{2}
\boldsymbol{\sigma}(\boldsymbol{x}) = J^{-1} \boldsymbol{P}(\boldsymbol{X}) \boldsymbol{F}(\boldsymbol{X})^T\,.
\end{equation}
In the above, $J = \mathrm{det}(\boldsymbol{F})$ is the determinant of $\boldsymbol{F}$.
- $\boldsymbol{S}(\boldsymbol{X})$ is the second Piola Kirchhoff stress tensor defined in reference configuration. The key property of $\boldsymbol{S}$ is that it is symmetric, and it is often used in place of $\boldsymbol{P}$. The first and second Piola stresses are related as follows: 
\begin{equation}\tag{3}
\boldsymbol{P} = \boldsymbol{F} \boldsymbol{S} \qquad \text{or} \qquad \boldsymbol{S} = \boldsymbol{F}^{-1} \boldsymbol{P},.
\end{equation}
- $\boldsymbol{b}(\boldsymbol{X})$ is the body force per unit volume defined on the reference configuration of solid. If $\bar{\boldsymbol{b}(\boldsymbol{x})}$ denotes the body force on current configuration of solid, we have $\boldsymbol{b}(\boldsymbol{X}) = J \bar{\boldsymbol{b}}(\boldsymbol{x})$.
- $\boldsymbol{t}(\boldsymbol{X})$ is the specified traction (force vector per unit area) at $\boldsymbol{X}$ on some part of the boundary, say $\Gamma_r \in \partial \Omega$. We have, for $\boldsymbol{X} \in \Gamma_r$, $\boldsymbol{t}(\boldsymbol{X}) = \boldsymbol{P}(\boldsymbol{X}) \boldsymbol{n}(\boldsymbol{X})$, where $\boldsymbol{n}(\boldsymbol{X})$ is the unit outward normal at $\boldsymbol{X}$ on the boundary.

To model the magnetic effects, we introduce some new notations
- $\phi: \Omega \to [0,1]$ volume fraction of magnetic materials. At any point $\boldsymbol{X}$, $\phi(\boldsymbol{X}) = 0$ means zero magnetic material and $1$ means full magnetic material. $\phi$ is nondimensional.
- $\boldsymbol{B}^{\text{r}}: \Omega \to \mathbb{R}^3$ Remanant magnetization flux density. We assume $\boldsymbol{B}^{\text{r}} = \phi \bar{\boldsymbol{B}}^{\text{r}}$, where $\bar{\boldsymbol{B}}^{\text{r}}$ is a constant vector. The units of $\vert\boldsymbol{B}^{\text{r}}\vert$ is Tesla (N/m/A).
- $\boldsymbol{B}^{\text{a}}$ applied constant magnetic flux density vector. 

We define the three invariants $\mathcal{I}_i(\boldsymbol{C})$ as follows
\begin{equation}\tag{4}
\mathcal{I}_1(\boldsymbol{C}) = \mathrm{tr}(\boldsymbol{C})\,, \quad \mathcal{I}_2(\boldsymbol{C}) = \frac{1}{2}\left[\left(\mathrm{tr}(\boldsymbol{C})\right)^2 - \mathrm{tr}\left( \boldsymbol{C}\right)^2 \right]\,, \quad \mathcal{I}_3(\boldsymbol{C}) = \mathrm{det}(\boldsymbol{C})\,,
\end{equation}
where $\mathrm{tr}$ and $\mathrm{det}$ are trace and determinant operators.

The equilibrium configuration of the body under external traction $\boldsymbol{t}$ on the boundary and body force $\boldsymbol{b}$ is the one where the displacement $\boldsymbol{u}$ satisfies the following boundary value problem:
\begin{align}\tag{5}
-\nabla \cdot \boldsymbol{P} &= \boldsymbol{b} && \qquad \text{in }\;\Omega, \\
 \boldsymbol{F} \boldsymbol{P}^T &= \boldsymbol{P} \boldsymbol{F}^T && \qquad \text{in }\;\Omega, \\
\boldsymbol{u} &= \boldsymbol{0} && \qquad \text{on }\;\Gamma_{l}, \\
\boldsymbol{P}\boldsymbol{n} &= \boldsymbol{t} && \qquad \text{on }\;\Gamma_{r}, \\
\boldsymbol{P}\boldsymbol{n} &= \boldsymbol{0} && \qquad \text{on }\;\partial \Omega - (\Gamma_{r}\cup \Gamma_{l}), \\
\end{align}
where the second equation is the balance of angular momentum, and $\partial \Omega - (\Gamma_{r}\cup \Gamma_{l})$ is the boundary of the domain excluding the left and right surfaces (at $x=0$ and $x=L$).

### Constitutive law

Constitutive law relates the stress, the first Piola Kirchhoff stress tensor $\boldsymbol{P}$, to the strain, the Green Strain tensor $\boldsymbol{F}$. Or we can write the relation between $\boldsymbol{S}$ and $\boldsymbol{E}$, and using the relations between various quantities, we can get the relation between $\boldsymbol{P}$ and $\boldsymbol{F}$.

#### Strain energy density
There are other versions of neo Hookean models. In general, we can consider a strain energy density function $W = W(\boldsymbol{X}, \boldsymbol{F}, \boldsymbol{B}^{\text{a}})$ that depends on the material point $\boldsymbol{X}$, deformation gradient $\boldsymbol{F}$, and external magnetic flux density $\boldsymbol{B}^{\text{a}}$ (dependence on external magnetic flux density is specific to magnetic soft materials). The first Piola stress is given by
\begin{equation}\tag{6}
\boldsymbol{P} = \frac{\partial W}{\partial \boldsymbol{F}}\,,
\end{equation}
and the second Piola stress is given by
\begin{equation}\tag{7}
\boldsymbol{S} = \frac{\partial W}{\partial \boldsymbol{E}} = 2\frac{\partial W}{\partial \boldsymbol{C}}\,.
\end{equation}
Using the above two equations and the relation $\boldsymbol{P} = \boldsymbol{F} \boldsymbol{S}$, we have
\begin{equation}\tag{8}
\boldsymbol{P} = \boldsymbol{F} \frac{\partial W}{\partial \boldsymbol{E}} = 2 \boldsymbol{F} \frac{\partial W}{\partial \boldsymbol{C}}\,.
\end{equation}

**Compressible neo Hookean Material Model with Magnetostatics energy** We assume the total strain energy is sum of elastic and magnetostatics energy. For elastic energy, we consider:
\begin{equation}\tag{9}
W_{\text{elastic}} = \frac{\mu}{2}\left[J^{-2/3}\mathcal{I}_1(\boldsymbol{C}) - 3\right] + \frac{K}{2} (J - 1)^2 \,,
\end{equation}
where recall that $J = \mathrm{det}(\boldsymbol{F})$, and $\mu$ and $K$ are shear and bulk modulii. Magnetostatics energy is given by
\begin{equation}\tag{10}
W_{\text{magnetic}} = -\frac{1}{\mu_0} \left(\boldsymbol{F} \boldsymbol{B}^{\text{r}} \right)\cdot \boldsymbol{B}^{\text{a}} \,,
\end{equation}
where $\mu_0 = 1.256 \times 10^{-6}$ N/A^2 is the magnetic permeability. Total energy is $W = W_{\text{elastic}} + W_{\text{magnetic}}$. 

To compute the stress $\boldsymbol{P}$, we will use the formula $\boldsymbol{P} = \frac{\partial W}{\partial \boldsymbol{F}}$ together with Fenics autodifferentiation feature to compute the derivatives. In the above expression, the first term models the energy contribution due to isochoric (volume-preserved) deformation and the second term models the energy contribution due to volumetric deformation. 


### Variational formulation

Let $V$ be the appropriate function space for the displacement (functions in $V$ are vector-valued functions). Due to homogeneous Dirichlet boundary condition, the trial and test functions belong to the same function space $V$. 

Multiplying the strong form of the problem by test function $\boldsymbol{v}\in V$ and integrating it over the domain $\Omega$ gives:
\begin{equation}\tag{11}
-\int_{\Omega} \left(\nabla \cdot \boldsymbol{P}\right) \cdot \boldsymbol{v} \,d\boldsymbol{X} = \int_{\Omega} \boldsymbol{b} \cdot \boldsymbol{v} \,d\boldsymbol{X} .
\end{equation}
The term on the left can also be written using integration by parts:
\begin{equation}\tag{12}
-\int_{\Omega} \left(\nabla \cdot \boldsymbol{P}\right) \cdot \boldsymbol{v} \,d\boldsymbol{X} = -\int_{\partial \Omega} \left(\boldsymbol{P}\boldsymbol{n}\right)\cdot \boldsymbol{v} \,d\boldsymbol{X} + \int_{\Omega} \boldsymbol{P} \boldsymbol{\colon} \nabla \boldsymbol{v} \,d\boldsymbol{X} = -\int_{\Gamma_{r}} \boldsymbol{t}\cdot \boldsymbol{v} \,d\boldsymbol{X} + \int_{\Omega} \boldsymbol{P} \boldsymbol{\colon} \nabla \boldsymbol{v} \,d\boldsymbol{X}.
\end{equation}
Using the above, the variational problem can be stated as follows:
\begin{equation}\tag{13}
\text{find }\;\boldsymbol{u}\in V \;\text{such that }\qquad \underbrace{\int_{\Omega} \boldsymbol{P} \boldsymbol{\colon} \nabla \boldsymbol{v} \,d\boldsymbol{X}}_{=: a(\boldsymbol{u}; \boldsymbol{v})} = \underbrace{\int_{\Omega} \boldsymbol{b} \cdot \boldsymbol{v} \,d\boldsymbol{X} + \int_{\Gamma_{r}} \boldsymbol{t}\cdot \boldsymbol{v} \,d\boldsymbol{X}}_{=:l(\boldsymbol{v})} \qquad \text{for all }\; \boldsymbol{v}\in V,
\end{equation}
where, $\boldsymbol{P}$ is related to $\boldsymbol{u}$ via the constitutive law, and $a(\cdot; \cdot)$ and $l(\cdot)$ are semilinear (not bilinear! as in the case of linear elasticity) and linear forms. 

# Units
- Lenth = mm, Time = s, Mass = kg, Charge = kC (kilo Coloumb) 
- Force = kg*mm/s^2 = mN (mili Newton, i.e., 10^(-3) N)
- Current = Charge/Time = kA (kilo Ampere)
- Stress = Force/Length^2 = kPa (kilo Pascal, i.e., 10^3 Pa)
- Magnetic flux density = Force/Length/Current = mT (mili Tesla)
- Permeability = $1.256\times 10^{-6}$ N/A^2 = $1.256\times 10^{-6} \times 10^9$ (mN)/(kA)^2

### Geometrical, material, and external loading parameters and functions

In what follows, we let $L = 17.2$ mm, $W = 0.84$ mm, and $H = 5$ mm. We assume no body force, i.e., $\boldsymbol{b} = \boldsymbol{0}$. The traction loading $\boldsymbol{t} = (t_x, t_y, t_z)$ on the right boundary $\Gamma_{r}$ is given by, for all $\boldsymbol{x} = (x, y, z) \in \Gamma_{r}$:
\begin{equation}\tag{14}
t_x(\boldsymbol{x}) = 0, \quad t_y(\boldsymbol{x}) =  0, \quad t_z(\boldsymbol{x}) = f_{bend}\,.
\end{equation}
We may take $f_{bend} = 0$ or $f_{bend} = 1$ kPa.

As for the material properties, we take Shear modulus $G = 303$ kPa. To model the materials as incompressible solid, we consider $\nu = 0.48$ (recall that truly incompressible solid will have $\nu = 0.5$) and compute bulk modulus using $K = 2G(1+\nu)/(1-2\nu)$.

Magnetic properties: maximum applied magnetic flux density in y-direction is $10$ mT, remanant magnetic flux density $\bar{\boldsymbol{B}}^{\text{r}} = [143, 0, 0]$ (mT), and volume fraction of magnetic materials is assumed to be $0.2$ throughout the beam. 

# Results
<img src="results/magnetoelastic_results.png" style="width:600px;">

### Fenics implementation

We start by loading the relevant packages:

In [1]:
import dolfinx
import numpy as np
import sys
import os

import pyvista


from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

from petsc4py import PETSc

# specific functions from dolfinx modules
from dolfinx import fem, mesh, io, plot, log
from dolfinx.fem import (Constant, dirichletbc, Function, functionspace, Expression )
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.io import VTXWriter
from dolfinx.nls.petsc import NewtonSolver

# specific functions from ufl modules
import ufl
from ufl import (TestFunctions,TestFunction, TrialFunction, Identity, grad, det, 
                 variable, div, dev, inv, tr, sqrt, conditional ,\
                 gt, dx, inner, derivative, dot, ln, \
                 split, outer, cos, acos, lt, eq, ge, le, exp, diff)

# basix finite elements
import basix
from basix.ufl import element, mixed_element, quadrature_element

# for plotting
import matplotlib.pyplot as plt
plt.close('all')

from datetime import datetime

log.set_log_level(log.LogLevel.WARNING)

Next, we set values of different parameters in the simulation:

In [2]:
## Parameters
# geometry 
omega_L, omega_W, omega_H = 17.2, 0.84, 5.0
num_elem_L, num_elem_W, num_elem_H = 20, 4, 4

# material
G_val = 303.0
G_model = 'Default'
# K_val = 1.e+3*G_val
nu = 0.48
K_val = 2*G_val*(1+nu)/(3*(1-2*nu))
phi_val = 0.2
B_app_max = 10.0
N_steps = 10
B_rem_val = 143.0
mu_0_val = 1.256e+3 # mN/(kA)^2
f_bend_max = 0.0

In [3]:
# Mesh and FE settings
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0,0.0,0.0], [omega_L, omega_W, omega_H]], [num_elem_L, num_elem_W, num_elem_H], mesh.CellType.tetrahedron)

# spatial dimension of the problem
d = domain.geometry.dim

# elements
UvecDG0 = element("DG", domain.basix_cell(), 0, shape=(3,))
UvecLag1 = element("Lagrange", domain.basix_cell(), 1, shape=(3,))
UvecLag2 = element("Lagrange", domain.basix_cell(), 2, shape=(3,))
UDG1 = element("DG", domain.basix_cell(), 1)
ULag1 = element("Lagrange", domain.basix_cell(), 1)
Uquad0 = quadrature_element(domain.basix_cell(), degree=2, scheme="default")

# function spaces
VvecDG0 = fem.functionspace(domain, UvecDG0)
VvecLag1 = fem.functionspace(domain, UvecLag1)
VvecLag2 = fem.functionspace(domain, UvecLag2)
VDG1 = fem.functionspace(domain, UDG1)
VLag1 = fem.functionspace(domain, ULag1)

In [4]:
# Displacement and test function
u = Function(VvecLag2, name = 'u')
v = TestFunction(VvecLag2)

# magnetig volume fraction (use DG so it can be different on different elements)
phi = Function(VDG1, name = 'phi')
phi.interpolate(lambda x: np.full((x.shape[1],), phi_val))

# Remanent magnetization flux density
def mag_flux_density(x):
    f = np.zeros((domain.geometry.dim, x.shape[1]), dtype=np.float64)
    f[0, :] = B_rem_val
    return f

B_rem = Function(VvecDG0, name = 'B_rem')
B_rem.interpolate(mag_flux_density)

In [5]:
G0 = Constant(domain, PETSc.ScalarType(G_val))
K = Constant(domain, PETSc.ScalarType(K_val))

def G_effective_Guth(phi):
    return G0*(1 + 2.5*phi + 14.1*phi**2)

def G_effective_Kerner(phi):
    A_factor = 15 * (1 - 0.5) / (8 - 10 * 0.5)
    return G0*(1 + A_factor*phi/(1 - phi))

def G_effective_Mooney(phi):
    return G0*exp(2.5*phi/(1-1.35*phi))

def G_effective_default(phi):
    return G0

G_effective = {
        'Guth': G_effective_Guth,
        'Kerner': G_effective_Kerner,
        'Mooney': G_effective_Mooney,
        'Default': G_effective_default,
    }

B_app = Constant(domain, PETSc.ScalarType((0., 0., 0.)))

mu0 = Constant(domain, PETSc.ScalarType(mu_0_val))

# body force
b = Constant(domain, PETSc.ScalarType((0., 0., 0.)))

# traction
## for complex traction
# f_bend = Constant(domain,PETSc.ScalarType(0.))

# class traction_expr():
#     def __init__(self, f_bend):
#         self.f_bend = f_bend

#     def __call__(self, x):
#         vals = np.zeros_like(x)
#         vals[1,:] = self.f_bend
#         return vals
    
# t_expr = traction_expr(f_bend)
# t_field = Function(VvecLag2)
# t_field.interpolate(t_expr)

## for simple constant traction
traction = Constant(domain, PETSc.ScalarType((0., 0., 0.)))

Next, define boundaries for boundary conditions:

In [6]:
# define edges/surfaces of beam
def xBot(x):
    return np.isclose(x[0], 0)
def xTop(x):
    return np.isclose(x[0], omega_L)
def yBot(x):
    return np.isclose(x[1], 0)
def yTop(x):
    return np.isclose(x[1], omega_W)
def zBot(x):
    return np.isclose(x[2], 0)
def zTop(x):
    return np.isclose(x[2], omega_H)
    
# mark the sub-domains
boundaries = [(1, xBot),(2,xTop),(3,yBot),(4,yTop),(5,zBot),(6,zTop)]

# build collections of facets on each subdomain and mark them appropriately.
facet_indices, facet_markers = [], [] 
fdim = domain.topology.dim - 1
for (marker, locator) in boundaries:
    facets = mesh.locate_entities(domain, fdim, locator) 
    facet_indices.append(facets) 
    facet_markers.append(np.full_like(facets, marker)) 

# Format the facet indices and markers as required for use in dolfinx.
facet_indices = np.hstack(facet_indices).astype(np.int32)
facet_markers = np.hstack(facet_markers).astype(np.int32)
sorted_facets = np.argsort(facet_indices)
 
# Add these marked facets as "mesh tags" for later use in BCs.
facet_tags = mesh.meshtags(domain, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

# create connectivity between the 2D and 3D entities
domain.topology.create_connectivity(domain.topology.dim-1, domain.topology.dim)

Define the surface area measure for integration over the right boundary for the traction boundary condition:

In [7]:
# ds for integration over the surface
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tags, metadata={'quadrature_degree':2})

Implement Dirichlet boundary condition on displacement next:

In [8]:
# Bottom surface displacement degrees of freedom
Btm_dofs_u1 = fem.locate_dofs_topological(VvecLag2.sub(0), facet_tags.dim, facet_tags.find(1))
Btm_dofs_u2 = fem.locate_dofs_topological(VvecLag2.sub(1), facet_tags.dim, facet_tags.find(1))
Btm_dofs_u3 = fem.locate_dofs_topological(VvecLag2.sub(2), facet_tags.dim, facet_tags.find(1))

# Dirichlet BCs into one vector
bcs_0 = dirichletbc(0.0, Btm_dofs_u1, VvecLag2.sub(0)) 
bcs_1 = dirichletbc(0.0, Btm_dofs_u2, VvecLag2.sub(1)) 
bcs_2 = dirichletbc(0.0, Btm_dofs_u3, VvecLag2.sub(2)) 

bcs = [bcs_0, bcs_1, bcs_2]

Finally, we are ready to define the bilinear and linear forms associated with the boundary value problem on displacement:

In [9]:
# Identity tensor
I = variable(ufl.Identity(d))

# Deformation gradient
F = variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = variable(F.T * F)

# Invariants of deformation tensors
Ic = variable(tr(C))
J = variable(det(F))

# get G
G = G_effective[G_model](phi)

# Elastic energy
W_elastic = (G / 2) * (J**(-2/3)*Ic - 3) + (K/2)*(J - 1)**2

# use log of J for compressible neo-Hookean model
# lamda = K - 2/3*G
# W_elastic = (G / 2) * (Ic - 3) - G * ufl.ln(J) + (lamda / 2) * (ufl.ln(J))**2

# Magnetic energy
W_magnetic = -(1/mu0) * phi* inner(F*B_rem, B_app)

# Total energy
W = W_elastic + W_magnetic

# First Piola-Kirchhoff stress tensor
P = diff(W, F)

In [10]:
# semilinear form
a = inner(grad(v), P)*dx

# linear form
L = inner(v, b)*dx + inner(v, traction)*ds(2)

# residual form
R = L - a

Compute post-processing quantities such as von-Mises stress that is defined as
\begin{equation}
\sigma_v = \sqrt{\frac{3}{2}\boldsymbol{\sigma}_{dev}\boldsymbol{\colon}\boldsymbol{\sigma}_{dev}} = \sqrt{\frac{3}{2}\sigma_{dev, ij} \sigma_{dev, ij}}\;, \quad \text{where}\quad  \boldsymbol{\sigma}_{dev} = \text{deviatoric stress} = \boldsymbol{\sigma} - \frac{\mathrm{tr}(\boldsymbol{\sigma})}{3}\boldsymbol{I} 
\end{equation}
where $\sigma = J^{-1} \boldsymbol{P}\boldsymbol{F}^T$.

In [11]:
# for visualization
u_vis = Function(VvecLag1, name = 'u')
# u_vis.interpolate(u)

phi_vis = Function(VLag1, name = 'phi')
B_rem_expr = Expression(phi*B_rem, VvecLag1.element.interpolation_points())
B_rem_vis = Function(VvecLag1, name = 'B_rem')
B_app_expr = Expression(B_app, VvecLag1.element.interpolation_points())
B_app_vis = Function(VvecLag1, name = 'B_app')

# cauchy stress from first Piola-Kirchhoff stress
sigma = P*F.T/J

# deviatoric stress
sigma_dev = P - (1./3)*tr(sigma)*Identity(d)  # deviatoric stress
vm = sqrt((3/2)*inner(sigma_dev, sigma_dev))
sigma_vm_expr = Expression(vm, VLag1.element.interpolation_points())
sigma_vm = Function(VLag1, name = 'von Mises stress')
# sigma_vm.interpolate(sigma_vm_expr)

# Compute magnitude of displacement
u_magnitude_expr = Expression(sqrt(dot(u, u)), VLag1.element.interpolation_points())
u_magnitude = Function(VLag1, name = 'magnitude(u)')
# u_magnitude.interpolate(u_magnitude_expr)

# interpolate W_elastic and W_magnetic into a function
W_elastic_expr = Expression(W_elastic, VLag1.element.interpolation_points())
W_magnetic_expr = Expression(W_magnetic, VLag1.element.interpolation_points())
W_elastic_vis = Function(VLag1, name = 'W_elastic')
W_magnetic_vis = Function(VLag1, name = 'W_magnetic')

# output results
results_folder = "fwd_result/"
os.makedirs(results_folder, exist_ok=True)
filename = results_folder + "/solution"
file_results = io.VTXWriter(domain.comm, filename + ".bp", 
                            [u_vis, phi_vis, B_rem_vis, B_app_vis, u_magnitude, sigma_vm, W_elastic_vis, W_magnetic_vis], 
                            engine="BP4")

def write_sim(t):
    phi_vis.interpolate(phi)
    B_rem_vis.interpolate(B_rem_expr)
    B_app_vis.interpolate(B_app_expr)
    sigma_vm.interpolate(sigma_vm_expr)
    u_magnitude.interpolate(u_magnitude_expr)
    u_vis.interpolate(u)
    W_elastic_vis.interpolate(W_elastic_expr)
    W_magnetic_vis.interpolate(W_magnetic_expr)

    file_results.write(t)



Since this is a nonlinear problem, we use Fenics in-built nonlinear solver. We start by creating a nonlinear problem and assign values to key solver parameters. 

In [12]:
problem = NonlinearProblem(R, u, bcs)

solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"

In [13]:
def get_norm(u):
    e = fem.form(ufl.inner(u, u) * ufl.dx)
    e_local = fem.assemble_scalar(e)
    e_global = domain.comm.allreduce(e_local, op=MPI.SUM)
    return e_global

In [14]:
log.set_log_level(log.LogLevel.INFO)

write_sim(0.0)

# reinitialize displacement
u.x.array[:] = 0.0
B_app.value[:] = 0.0
b.value[1] = 0.0 
traction.value[:] = 0.0

N = 10
for n in range(1, N_steps+1):
    B_app.value[1] = (n/N_steps) * B_app_max
    traction.value[1] = (n/N_steps) * f_bend_max
    # b.value[1] = -(n/N_steps) * 0.1
    # print(f"Time step {n}, B_app {B_app_const.value}, b {b.value}")

    # solve the problem
    num_its, converged = solver.solve(u)
    assert(converged)

    # write
    write_sim(n)

    # compute norms of W_elastic and W_magnetic and print them
    W_elastic_norm = get_norm(W_elastic_vis)
    W_magnetic_norm = get_norm(W_magnetic_vis)

    u_mag_max = u_magnitude.x.array.max()
    u_mag_min = u_magnitude.x.array.min()

    print("\n")
    print("+"*50)
    print(f"Time step {n}, Number of iterations {num_its}, B_app {B_app.value[1]}, W_elastic {W_elastic_norm}, W_magnetic {W_magnetic_norm}, u_mag_max {u_mag_max}, u_mag_min {u_mag_min}")
    print("+"*50)
    print("\n")


[2025-06-12 18:15:06.226] [info] PETSc Krylov solver starting to solve system.
[2025-06-12 18:15:08.332] [info] PETSc Krylov solver starting to solve system.
[2025-06-12 18:15:09.257] [info] Newton iteration 2: r (abs) = 0.4102563694407545 (tol = 1e-08), r (rel) = 0.021802129229431075 (tol = 1e-08)
[2025-06-12 18:15:10.371] [info] PETSc Krylov solver starting to solve system.
[2025-06-12 18:15:11.288] [info] Newton iteration 3: r (abs) = 0.010550296983955465 (tol = 1e-08), r (rel) = 0.0005606712177720161 (tol = 1e-08)
[2025-06-12 18:15:12.325] [info] PETSc Krylov solver starting to solve system.
[2025-06-12 18:15:13.213] [info] Newton iteration 4: r (abs) = 0.00092283495337393 (tol = 1e-08), r (rel) = 4.904193672439725e-05 (tol = 1e-08)
[2025-06-12 18:15:14.241] [info] PETSc Krylov solver starting to solve system.
[2025-06-12 18:15:15.102] [info] Newton iteration 5: r (abs) = 2.204990074304796e-08 (tol = 1e-08), r (rel) = 1.1717911562259932e-09 (tol = 1e-08)
[2025-06-12 18:15:15.102] [

### Plotting results in paraview
To plot, open a paraview app and drag the folder `solution.bp` in the box `Pipieline Browser` on left side of paraview app. Next, select the `Warp by Vector` from the `Filters` menu to add displacement to the reference configuration to get the current configuration. Next, select the `von Mises stress` from the plot field selector. 

<img src="results/magnetoelastic_results.png" style="width:600px;">
